# Remez Algorithm

We are working backwards from
[github.com/DKenefake/OptimalPoly](https://github.com/DKenefake/OptimalPoly/blob/main/remez_poly.py).

In [45]:
"""
Taken from https://github.com/DKenefake/OptimalPoly/blob/main/remez_poly.py
"""
from mpmath import mp
import numpy


def bisection_search(f, low:float, high:float):
    """
    A root finding method that does not rely on derivatives

    :param f: a function f: X -> R
    :param low: the lower bracket
    :param high: the upper limit bracket
    :return: the location of the root, e.g. f(mid) ~ 0
    """
    # flip high and low if out of order
    if f(high) < f(low):
        low, high = high, low

    # find mid point
    mid = .5 * (low + high)

    while True:

        # bracket up
        if f(mid) < 0:
            low = mid
        # braket down
        else:
            high = mid

        # update mid point
        mid = .5 * (high + low)

        # break if condition met
        if abs(high - low) < 10 ** (-(mp.dps / 2)):
            break

    return mid


def concave_max(f, low:float, high:float):
    """
    Forms a lambda for the approximate derivative and finds the root

    :param f: a function f: X -> R
    :param low: the lower bracket
    :param high: the upper limit bracket
    :return: the location of the root f'(mid) ~ 0
    """
    # create an approximate derivative expression
    scale = high - low

    h = mp.mpf('0.' + ''.join(['0' for i in range(int(mp.dps / 1.5))]) + '1') * scale
    df = lambda x: (f(x + h) - f(x - h)) / (2.0 * h)

    return bisection_search(df, low, high)

def chev_points(n:int, lower:float = -1, upper:float = 1):
    """
    Generates a set of chebychev points spaced in the range [lower, upper]
    :param n: number of points
    :param lower: lower limit
    :param upper: upper limit
    :return: a list of multipressison chebychev points that are in the range [lower, upper]
    """
    #generate chebeshev points on a range [-1, 1]
    index = numpy.arange(1, n+1)
    range_ = abs(upper - lower)
    return [(.5*(mp.cos((2*i-1)/(2*n)*mp.pi)+1))*range_ + lower for i in index]


def remez(func, n_degree:int, lower:float=-1, upper:float=1, max_iter:int = 10):
    """
    :param func: a function (or lambda) f: X -> R
    :param n_degree: the degree of the polynomial to approximate the function f
    :param lower: lower range of the approximation
    :param upper: upper range of the approximation
    :return: the polynomial coefficients, and an approximate maximum error associated with this approximation
    """
    # initialize the node points

    x_points = chev_points(n_degree + 2, lower, upper)

    A = mp.matrix(n_degree + 2)
    coeffs = numpy.zeros(n_degree + 2)

    # place in the E column
    mean_error = float('inf')

    for i in range(n_degree + 2):
        A[i, n_degree + 1] = (-1) ** (i + 1)

    for i in range(max_iter):

        # build the system
        vander = numpy.polynomial.chebyshev.chebvander(x_points, n_degree)

        for i in range(n_degree + 2):
            for j in range(n_degree + 1):
                A[i, j] = vander[i, j]

        b = mp.matrix([func(x) for x in x_points])
        l = mp.lu_solve(A, b)

        coeffs = l[:-1]

        # build the residual expression
        r_i = lambda x: (func(x) - numpy.polynomial.chebyshev.chebval(x, coeffs))

        interval_list = list(zip(x_points, x_points[1:]))
        #         interval_list = [[x_points[i], x_points[i+1]] for i in range(len(x_points)-1)]

        intervals = [upper]
        intervals.extend([bisection_search(r_i, *i) for i in interval_list])
        intervals.append(lower)

        extermum_interval = [[intervals[i], intervals[i + 1]] for i in range(len(intervals) - 1)]

        extremums = [concave_max(r_i, *i) for i in extermum_interval]

        extremums[0] = mp.mpf(upper)
        extremums[-1] = mp.mpf(lower)

        errors = [abs(r_i(i)) for i in extremums]
        mean_error = numpy.mean(errors)

        if numpy.max([abs(error - mean_error) for error in errors]) < 0.000001 * mean_error:
            break

        x_points = extremums

    return [float(i) for i in numpy.polynomial.chebyshev.cheb2poly(coeffs)], float(mean_error)

def c_code_gen(data_type, name, poly_coeffs, comments = None):
    method_string = f'{data_type} {name} ({data_type} x)' + '{\n'
    
    if comments is not None:
        method_string += '\t// ' + str(comments) + ' \n\n'
    
    data_type_converter = '' if data_type == 'double' else 'f'
    
    method_string += '\n'.join([f'\tconst {data_type} a_{i} = {str(val) + data_type_converter};' for i, val in enumerate(poly_coeffs)])
    
    horner = 'return a_0+'
    for i in range(len(poly_coeffs)-2):
        horner += f'x*(a_{i+1} +' 
    horner += f'x*a_{len(poly_coeffs)-1}' + ')'*(len(poly_coeffs)-2) + ';\n}'
    
    return method_string + '\n \t' + horner

## Step by Step

### Chebyshev Points

In [2]:
from mpmath import mp
import numpy as np

In [17]:
n = 5
lower = -1
upper = 1

index = np.arange(1, n+1)
index

array([1, 2, 3, 4, 5])

In [20]:
range_ = abs(upper - lower)

# Chebyshev polynomial of degree n = 5.
T_5 = [(.5*(mp.cos((2*i-1)/(2*n)*mp.pi)+1))*range_ + lower for i in index]
T_5

[mpf('0.95105651629515364'),
 mpf('0.58778525229247314'),
 mpf('0.0'),
 mpf('-0.58778525229247303'),
 mpf('-0.95105651629515353')]

In [21]:
mp.cos(mp.pi / 10), np.cos(np.pi / 10), mp.cos(np.pi / 10)

(mpf('0.95105651629515353'),
 np.float64(0.9510565162951535),
 mpf('0.95105651629515353'))

In [22]:
# 0.951056516295153 64
# 0.951056516295153 53
mp.mpf('0.95105651629515364') - mp.cos(mp.pi / 10)

mpf('1.1102230246251565e-16')

The Chebyshev roots for the Chebyshev polynomial of degree 5 are
$$
\cos\left( \frac{  \pi}{10} \right),
\cos\left( \frac{3 \pi}{10} \right),
\cos\left( \frac{5 \pi}{10} \right),
\cos\left( \frac{7 \pi}{10} \right),
\cos\left( \frac{9 \pi}{10} \right)
$$

In [39]:
for idx, coef in enumerate([1, 3, 5, 7, 9]):  # Coefficients of the Chebyshev roots.
    cheb_root = mp.cos( (coef/10)*mp.pi )
    diff = float( T_5[idx] - cheb_root )
    print(f"{diff:.16f}")

0.0000000000000001
0.0000000000000000
-0.0000000000000001
0.0000000000000000
0.0000000000000000


The general formula for the chebyshev roots, once you stretch them to cover the range $[a, b]$
from their original domain of $[-1, 1]$
and you translate their center of mass from $0$ to the midpoint of $[a, b]$, is
$$
\frac{b-a}{2} \cos \left( \frac{\text{odd }\pi}{2n} \right)
    + \frac{b+a}{2}
$$

See "Numerical Analysis" 2nd Ed. by Timothy Sauer, Section 3.3.3, Page 162.

Another way of expressing the above is to say that on the interval $[a, b]$,
$$
x_i = \frac{b+a}{2}
    + \frac{b-a}{2} 
        \cos \left( \frac{(2i - 1)\pi}{2n} \right)
$$
for $i = 1, \dots, n$.

Or
$$
x_i = \frac{b-a}{2} 
        \left[ 
            \cos \left( \frac{(2i - 1)\pi}{2n} \right)
            + 1
        \right]
    + a
$$

We show this other expression because people use it and its good to not get confused by it.
This is actually the formula we started using.
It is easier to see it once we reorganize it a tad.

In [42]:
n = 5
lower = -1
upper = 1
range_ = abs(upper - lower)

[
    (
        .5*(
            mp.cos((2*i-1)/(2*n)*mp.pi) + 1
        )
    )*range_ + lower for i in index
]

[mpf('0.95105651629515364'),
 mpf('0.58778525229247314'),
 mpf('0.0'),
 mpf('-0.58778525229247303'),
 mpf('-0.95105651629515353')]

### Linear System

In [46]:
n_degree = 3
mp.matrix(n_degree + 2)

matrix(
[['0.0', '0.0', '0.0', '0.0', '0.0'],
 ['0.0', '0.0', '0.0', '0.0', '0.0'],
 ['0.0', '0.0', '0.0', '0.0', '0.0'],
 ['0.0', '0.0', '0.0', '0.0', '0.0'],
 ['0.0', '0.0', '0.0', '0.0', '0.0']])

In [47]:
numpy.zeros(n_degree + 2)

array([0., 0., 0., 0., 0.])

In [50]:
# This operation iterates through all the rows, filling in an alternating
# value for the last column `n_degree + 1` (remember we count from 0).
A = mp.matrix(n_degree + 2)
for i in range(n_degree + 2):
    A[i, n_degree + 1] = (-1) ** (i + 1)
A

matrix(
[['0.0', '0.0', '0.0', '0.0', '-1.0'],
 ['0.0', '0.0', '0.0', '0.0', '1.0'],
 ['0.0', '0.0', '0.0', '0.0', '-1.0'],
 ['0.0', '0.0', '0.0', '0.0', '1.0'],
 ['0.0', '0.0', '0.0', '0.0', '-1.0']])

In [65]:
# See
# https://numpy.org/doc/stable/reference/generated/numpy.polynomial.chebyshev.chebvander.html
# and
# https://en.wikipedia.org/wiki/Vandermonde_matrix
# and
# https://en.wikipedia.org/wiki/Chebyshev_polynomials

n_degree = 3
lower = -1
upper = 1
x_points = chev_points(n_degree + 2, lower, upper)
vander = numpy.polynomial.chebyshev.chebvander(x_points, n_degree)
vander.shape

(5, 4)

The numpy page calls the output of `chebvander` a pseudo-vandermonde matrix because the matrix is not generated via a geometric progression ($x^0, x^1, x^2, x^3, \dots$), but its generated by using the Chebyshev polynomials of the first kind:

$$
T_0(x) = 1
$$
$$
T_1(x) = x
$$
$$
T_{n+1}(x) = 2x T_n(x) - T_{n-1}(x)
$$

If we keep this going, 
$$
T_2(x) = 2T_1(x) T_1(x) - T_0(x) = 2(T_1)^2 - 1
$$
$$
T_3(x) = 2T_1(x) T_2(x) - T_1(x) = 2 T_1 \left( 2(T_1)^2 - 1 \right) - T_1
    = 2(T_1)^3 - 2T_1 - T_1 = 2(T_1)^3 - 3T_1
$$

In Python this would look a bit like
```python
ideg = pu._as_int(deg, "deg")

x = np.array(x, copy=None, ndmin=1) + 0.0
dims = (ideg + 1,) + x.shape
dtyp = x.dtype
v = np.empty(dims, dtype=dtyp)
# Use forward recursion to generate the entries.
v[0] = x*0 + 1
if ideg > 0:
    x2 = 2*x
    v[1] = x
    for i in range(2, ideg + 1):
        v[i] = v[i-1]*x2 - v[i-2]
```
The above code comes from
[github.com/numpy/polynomial/chebyshev.py#L1391-L1441](https://github.com/numpy/numpy/blob/2f7fe64b8b6d7591dd208942f1cc74473d5db4cb/numpy/polynomial/chebyshev.py#L1391-L1441).

In [56]:
x_points, vander

([mpf('0.95105651629515364'),
  mpf('0.58778525229247314'),
  mpf('0.0'),
  mpf('-0.58778525229247303'),
  mpf('-0.95105651629515353')],
 array([[mpf('1.0'), mpf('0.95105651629515364'),
         mpf('0.80901699437494767'), mpf('0.58778525229247358')],
        [mpf('1.0'), mpf('0.58778525229247314'),
         mpf('-0.30901699437494745'), mpf('-0.95105651629515364')],
        [mpf('1.0'), mpf('0.0'), mpf('-1.0'), mpf('0.0')],
        [mpf('1.0'), mpf('-0.58778525229247303'),
         mpf('-0.30901699437494767'), mpf('0.95105651629515364')],
        [mpf('1.0'), mpf('-0.95105651629515353'),
         mpf('0.80901699437494723'), mpf('-0.5877852522924728')]],
       dtype=object))

In [67]:
[ i for i in range(2, n_degree+2) ]

[2, 3, 4]

In [68]:
x = x_points[0]
v_manual = np.zeros(n_degree+2)
v_manual[0] = 1.0
v_manual[1] = x
for i in range(2, n_degree+2):
    v_manual[i] = 2 * x * v_manual[i-1] - v_manual[i-2]

v_manual

array([1.        , 0.95105652, 0.80901699, 0.58778525, 0.30901699])

In [69]:
# Manually computing the first 3 values of the Chebyshev polynomial.
x_points[0]**0, x_points[0]**1, 2* x_points[0]**2 - 1.0

(mpf('1.0'), mpf('0.95105651629515364'), mpf('0.80901699437494767'))

In [70]:
# List the first row.
# Should have all the values of the Chebyshev polynomial of degree 5.
vander[0]

array([mpf('1.0'), mpf('0.95105651629515364'), mpf('0.80901699437494767'),
       mpf('0.58778525229247358')], dtype=object)

In [61]:
# List the first column,
vander[:,0]

array([mpf('1.0'), mpf('1.0'), mpf('1.0'), mpf('1.0'), mpf('1.0')],
      dtype=object)

#### np.moveaxis

Numpy's implementation of chebvander does this right as it returns.

In [77]:
x = np.array(
    [
        [1,  2,  3,  4,  5],
        [6,  7,  8,  9,  10],
        [11, 12, 13, 14, 15] ]
)
x

array([[ 1,  2,  3,  4,  5],
       [ 6,  7,  8,  9, 10],
       [11, 12, 13, 14, 15]])

In [78]:
np.moveaxis(x, 0, -1)

array([[ 1,  6, 11],
       [ 2,  7, 12],
       [ 3,  8, 13],
       [ 4,  9, 14],
       [ 5, 10, 15]])